In [1]:
import dates_generation as dg
import os.path
import pandas as pd

Dates generation


In [2]:
def date_generation_process(month, year, holidays_list):
    holidays_list = dg.convert_string_to_date(holidays)
    days_list = dg.weekdays_list(month, year)
    business_days_list = dg.generate_date_from_list(month, year, days_list)
    real_business = dg.extract_holidays_from_weekdays(business_days_list, holidays_list)
    list_of_dates = dg.create_list_of_dates(real_business)
    dictionary = dg.create_dataframe_dictionary(list_of_dates)
    return dictionary

In [3]:
holidays = ["05-05-23", "04-05-23"]

In [4]:
dates_dictionary = date_generation_process(5, 2023, holidays)

2023-05-01 00:00:00
2023-05-02 00:00:00
2023-05-03 00:00:00
2023-05-06 00:00:00
2023-05-08 00:00:00
2023-05-09 00:00:00
2023-05-10 00:00:00
2023-05-11 00:00:00
2023-05-12 00:00:00
2023-05-13 00:00:00
2023-05-15 00:00:00
2023-05-16 00:00:00
2023-05-17 00:00:00
2023-05-18 00:00:00
2023-05-19 00:00:00
2023-05-20 00:00:00
2023-05-22 00:00:00
2023-05-23 00:00:00
2023-05-24 00:00:00
2023-05-25 00:00:00
2023-05-26 00:00:00
2023-05-27 00:00:00
2023-05-29 00:00:00
2023-05-30 00:00:00
2023-05-31 00:00:00


In [5]:
dates_dictionary["01-05-2023"]

,articulo,descripcion,chess,wms,fisico


Dataframe base ABC clasification


In [6]:
import dataframe_class_base as dcb

In [7]:
# read ABC base file
fullpath = os.path.abspath(".//abc_mayo_23.xlsx")

df_clasificacion = pd.read_excel(fullpath, sheet_name="Hoja1")

In [8]:
df_abc = dcb.cantidad_segun_abc(df_clasificacion)

In [9]:
df_abc

,articulo,descripcion,clasificacion,cantidad
0,7038,BRAHMA - X12 1000RET#,A,3
1,25668,PEPSI NF PET X6 500CC,A,3
2,7634,BRAHMA - 4X6 LAT 473,A,3
3,25644,7-UP NF PET X6 500CC,A,3
4,7026,QUILMES - X12 1000RET,A,3
...,...,...,...,...
318,78068,STA MAZO DE CARTAS BUDWEISER,C,1
320,78131,"VINO MALBEC MISTERIO X6 0,750 L",C,1
321,78141,TOSTADAS DE ARROZ MOLINOS ALA X24 150GR,C,1
325,78233,YERBA MATE CACHAMATE X12 500GR,C,1


DB Operations


In [10]:
import sqlite_db_generation as sqlgen
import sqlite3
import math

In [11]:
sqlgen.create_db("abc_quantity.db", "abc")

In [12]:
sqlgen.insert_article_df_into_db(df_abc, "abc_quantity.db", "abc")

In [13]:
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")

In [14]:
sqlgen.remove_quant_for_article("abc_quantity.db", "abc",25668)

In [15]:
sqlgen.select_an_article("abc_quantity.db", "abc", 25668)

(25668, 'PEPSI NF PET X6 500CC', 'A', 2)


In [16]:
random_art = sqlgen.select_random_article("abc_quantity.db", "abc", "A")

In [17]:
sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", random_art[0])

True

In [46]:
def get_max_per_category_per_day(db_name, table_name, dates_dictionary, category):
    cnt = sqlite3.connect(db_name)
    business_days = len(dates_dictionary.keys())
    query = cnt.execute(
        f""" SELECT COUNT(*)
            FROM {table_name}
            WHERE CLASIFICACION = '{category}'; """
    )

    if(category == "A"):
        quantity = query.fetchone()[0] * 3
    elif(category == "B"):
        quantity = query.fetchone()[0] * 2
    else:
        quantity = query.fetchone()[0] * 1

    return math.ceil(quantity/business_days)
    

In [19]:
# def add_article_to_dataframe(cod,descrip,dictionary,dictionary_key):
#     if cod in dictionary[dictionary_key].values:
#         return
#     else:
#         df_aux = {'articulo': cod, 'descripcion': descrip, 'chess': "", 'wms': "", 'fisico': "" }
#         dictionary[dictionary_key] = dictionary[dictionary_key].append(df_aux, ignore_index = True)
#     return dictionary

In [58]:
def add_article_to_dataframe(cod,descrip,date_df):
    df_aux = {'articulo': cod, 'descripcion': descrip, 'chess': "", 'wms': "", 'fisico': "" }
    date_df = pd.concat([date_df, pd.DataFrame([df_aux])], ignore_index=True)
    return date_df

In [77]:
def check_article_already_in_df(cod, date_df):
    if cod in date_df.values:
        print(date_df.values)
        return True
    else:
        return False

In [71]:
a_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"A")
b_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"B")
c_articles_per_day = get_max_per_category_per_day("abc_quantity.db", "abc",dates_dictionary,"C")

In [106]:
def fill_date_A(date_df, maxA):
    contador_a = 0
    while(contador_a < maxA):
        if(sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", "A")):
            rnd_art = sqlgen.select_random_article("abc_quantity.db", "abc", "A")
            if(check_article_already_in_df(rnd_art[0],date_df)):
                print(rnd_art[0], "already in df")
                continue
            else:
                if(sqlgen.check_if_article_has_quantity("abc_quantity.db", "abc", rnd_art[0])):
                    date_df = add_article_to_dataframe(rnd_art[0],rnd_art[1],date_df)
                    sqlgen.remove_quant_for_article("abc_quantity.db", "abc",rnd_art[0])
                    contador_a+=1
                else:
                    continue
        else:
            break
    return date_df

In [84]:
#reset dates dictionary
dates_dictionary = date_generation_process(5, 2023, holidays)

2023-05-01 00:00:00
2023-05-02 00:00:00
2023-05-03 00:00:00
2023-05-06 00:00:00
2023-05-08 00:00:00
2023-05-09 00:00:00
2023-05-10 00:00:00
2023-05-11 00:00:00
2023-05-12 00:00:00
2023-05-13 00:00:00
2023-05-15 00:00:00
2023-05-16 00:00:00
2023-05-17 00:00:00
2023-05-18 00:00:00
2023-05-19 00:00:00
2023-05-20 00:00:00
2023-05-22 00:00:00
2023-05-23 00:00:00
2023-05-24 00:00:00
2023-05-25 00:00:00
2023-05-26 00:00:00
2023-05-27 00:00:00
2023-05-29 00:00:00
2023-05-30 00:00:00
2023-05-31 00:00:00


In [85]:
sqlgen.reset_database_quantity_for_article("abc_quantity.db", "abc")

In [86]:
sqlgen.check_if_category_has_quantity("abc_quantity.db", "abc", "A")

True

In [108]:
for i in dates_dictionary:
    dates_dictionary[i] = fill_date_A(dates_dictionary[i],a_articles_per_day)

In [104]:
for i in dates_dictionary:
    print(i)

01-05-2023
02-05-2023
03-05-2023
06-05-2023
08-05-2023
09-05-2023
10-05-2023
11-05-2023
12-05-2023
13-05-2023
15-05-2023
16-05-2023
17-05-2023
18-05-2023
19-05-2023
20-05-2023
22-05-2023
23-05-2023
24-05-2023
25-05-2023
26-05-2023
27-05-2023
29-05-2023
30-05-2023
31-05-2023


In [89]:
dates_dictionary.values()

dict_values([  articulo                   descripcion chess wms fisico
0    25194  MIRINDA NARANJA PET X6 500CC                 
1    25668         PEPSI NF PET X6 500CC                 
2     2218         QUILMES - 4X6 LAT 473                 
3    19337           7-UP PET X6 1500 NF                 ,   articulo                              descripcion chess wms fisico
0    25194             MIRINDA NARANJA PET X6 500CC                 
1    14933  ECO DE LOS ANDES SG PET X12 500 MIX 203                 
2    10580               PATAGO AMBER - X12 355 OW#                 
3     7038                    BRAHMA - X12 1000RET#                 ,   articulo                     descripcion chess wms fisico
0    11522  GATORADE FRU TRO * X6 PET 500#                 
1     1416   GATORADE MANZANA * X6 PET 500                 
2    19086            PEPSI PET X6 1500 NF                 
3     7478   MIRINDA NARANJA * X6 PET1500#                 ,   articulo                    descripcion chess w

In [101]:
dates_dictionary['26-05-2023']

,articulo,descripcion,chess,wms,fisico
0,7634,BRAHMA - 4X6 LAT 473,,,
1,2218,QUILMES - 4X6 LAT 473,,,
2,24880,STELLA ARTOIS PM RET X12 975CC,,,
3,14191,PEPSI BLACK PET X6 1500,,,
